# Part 2: Working with Dirty PDF and CSV Files.
## Issues Covered
* Unstructured text and PDFs
* Common problems with ill-formatted (i.e., non-relational) CSV files 
* Code reuse and reproducible analyses

## Practices Demonstrated
* Scraping tabular data from PDF files with [Tabula](https://tabula.technology/)
* Use of a custom module to make code reusable
* Defensive programming: preventing bugs *before* they happen
* Decomposition into modules to simplify logic and improve testability

## Preliminaries: Scraping PDFs with Tabula
__1. First, let's experience the unique challenge of pulling data from PDF files.__  
Take a look at the Fairfield University's [Spring 2018 Undergraduate Course Booklet](201801CourseBooklet.pdf). You'll see lots of pseudo-tabular text, where data is organized into rows and columns. However, when you try to copy text from a table like this ...
![Copy from PDF](img/P2_Copy_from_PDF.png)  
and then paste it into a text file you get this:  
![Paste from PDF](img/P2_Paste_from_PDF.png)  
Whoah, where'd the table go? Why is everything all jumbled? The dirty little secret of PDF files is that they are actually designed to minimize the work of displaying printed pages, not for us to use as data. The order in which the text is stored in the file is irrelevant as long as the physical location on the page is known. As far as PDF writers care, text is just shapes on a page. So, it's kind of random where everything ends up within the physical PDF file.

__2. In this exercise we are going to extract data from the course booklet into a CSV file and then clean it up using standard Python.__  
For the first part we are going to use [Tabula](https://tabula.technology/), which promises to be just the right tool for the job:
> **How Can Tabula Help Me?**    
If you’ve ever tried to do anything with data provided to you in PDFs, you know how painful it is — there's no easy way to copy-and-paste rows of data out of PDF files. Tabula allows you to extract that data into a CSV or Microsoft Excel spreadsheet using a simple, easy-to-use interface. Tabula works on Mac, Windows and Linux.
>
>**Who Uses Tabula?**    
Tabula is used to power investigative reporting at news organizations of all sizes, including ProPublica, The Times of London, Foreign Policy, La Nación (Argentina), The New York Times and the St. Paul (MN) Pioneer Press.
>
>Grassroots organizations like SchoolCuts.org rely on Tabula to turn clunky documents into human-friendly public resources.
>
>And researchers of all kinds use Tabula to turn PDF reports into Excel spreadsheets, CSVs, and JSON files for use in analysis and database applications.

**So why do we need Python if Tabula creates CSV files?** Even with all of the smarts Tabula provides, it still has to make a lot of guesses about the structure of the original table, the most problematic of which is picking out the column breaks. In PDFs there are no tabs or spaces. Those are just whitespace between characters. So, is a given bit of whitespace long enough to indicate a new column or is it just a normal space within the column text? It's almost impossible to tell sometimes and Tabula sometimes gets it wrong. The resulting CSV files may then have extra commas (column separators) in some places and be missing commas in others. Unfortunately for us, even one misplaced or missing comma can totally ruin our CSV file, so we have to clean the CSV before we can use it in our analyses. Fortunately, we the analysts (presumably) know what the original table was about and can usually make corrections manually. Or, better yet, use Python to do it for us. We just have to encode the correction rules and let Python do the rest.  

### You are encouraged to install and try out Tabula as shown below. 
Doing is always better than watching. While you may get slightly different results, it will certainly help to visualize what Tabula is doing for you and how it might help you with your projects. 

__3. Tabula is a Java-based installable web app.__  
The app requires a recent version of Java to install. Once installed it operates as a (fairly spare but functional) web app on your `localhost` (i.e., a web server running on your computer), just like JupyterLab.
![Tabula Web App](img/P2_Tabula_Web_App.png)  
![Tabula Import Screen](img/P2_Tabula_Import_Manager.png)  

__4. Importing a PDF and Extracting Data.__  
From the main screen you can `Import` a PDF file on your hard drive and then `Extract Data`. Tabula will open the file and invite you to Select Tables.
![Select Tables](img/P2_Select_Tables.png)  
Select each table you want to scrape by dragging out boxes with your mouse. Or, if you prefer, let Tabula Autodetect the tables for you and then correct any erroneous selections yourself.    

In our case the tables we want start on page 13 of the printed booklet:
![Autoselect Tables](img/P2_Autoselect_Tables.png)  
It seems to be doing a decent job, though (as we'll see) it has problems when tables span page breaks. With a bit of fiddling with the selections, however, you can get it mostly right. It certainly beats retyping it yourself.  

Once the tables have been selected, use the Preview and Export Extracted Data to generate a CSV file. 
![Preview showing erroneous column breaks](img/P2_Erroneous_Column_Breaks.png)  
Oops, it looks like we found a bad column break. Can you spot it? Hmmm.  

Let's export to CSV and then fix it ourselves. 
![Export Button](img/P2_Export_Button.png)  
Before moving on, take a moment to study the finished [tabula-201801CourseBooklet.csv](https://github.com/christopherhuntley/ba505-docs/blob/master/Tutorials/CourseCatalogDemo/tabula-201801CourseBooklet.csv) file.

## Source Data Definition and Assessment

__1. Column Definitions. All are text except where noted.__  
- __`CRN`__: the course reference number for the section of the course; `CRN` is unique, retired from use for future offerings
- __`Subj`__: the subject area for the course (e.g., "BA" for Business Analytics)
- __`Course`__: The course number within the subject area (e.g., "0505"); note that course "numbers" are not actually numbers and can contain text codes
- __`Sec`__: the section ID for the class; within each semester each class section has a distinct `Sec`
- __`Title`__: the course title, which is usually drawn from the course catalog but can be customized for a specific section
- __`Creds`__: the number of course credits; like `Course` there may be text mixed in (e.g., "1-3")
- __`Days`__: the days of the week that the course meets (e.g., "TF" means Tuesdays and Fridays); exceptions like "online" classes that don't meet at all or "by appointment" only classes are marked accordingly.
- __`Time`__: the meeting time for the course, typically in `<start time>` - `<end time>` format; time is left blank when the class is online or the time is unknown`
- __`Instructor`__: the instructor name(s); the names are unique for any given semester (and possibly unique thereafter)
- __`Notes`__: a list of special designations like 'WDiv' with meanings explained in the course catalog

__2. Bug Detection.__  
All things considered, Tabula did a greate job of capturing the data, but there were a few bus we'll need to address before we can use the data:
- __In the very first line of the file, the `Creds` column is merged with the `Days` column. This is repeated elsewhere throughout the file.__ 
![P2_CSV_Col_Break_Bug](img/P2_CSV_Col_Break_Bug.png)
- __The column headers are repeated every so often (e.g., on line 12) and should be skipped.__
![P2_Redundant_Headers_Bug](img/P2_Redundant_Headers_Bug.png)
- __On lines 49 and 50 we see that sometimes Tabula misses the break between the `Instructor` and `Notes` columns.__
![P2_Instructor_Notes_Bug](img/P2_Instructor_Notes_Bug.png)
- __On lines 95-98 we see that `Days` and `Time` also collide sometimes.__
![P2_Days_Time_Bug](img/P2_Days_Time_Bug.png)
- __On lines 137-143, some classes have extra meeting days and times. The extras are listed on a row by themselves. This is not actually an extraction bug, but rather a quirk of the table design. We'll need to work around it nonetheless.__ 
![P2_Multiple_Timecodes](img/P2_Multiple_Timecodes.png)

## Cleaning up with Python: A Walkthrough of the Code
__1. The full source code is in the [course_schedules_tabula.py](https://github.com/christopherhuntley/ba505-docs/blob/master/Tutorials/CourseCatalogDemo/course_schedules_tabula.py) module.__  
In this notebook we will study each section, one at a time, to better understand the code. But first, please take a moment to study the file. While some things should be new to you, care has been taken to make it easier to read and follow along on your own:
- Comments are used liberally to explain what each block of code does. 
- Variables and functions are given descriptive names that indicate what they represent or do. 
- White space is used to separate sections from each other. 

Imagine if there were no comments and variables names like x, y, z. Would you still be able to make sense of the code? Even for an experienced coder, reading code like that is hard.   

__2. The file header.__  
The top of the module includes a special comments intended to help us use the code. 

The first two lines are actually for running the file as a `bash` script in MacOS X:
```python
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
```
This tells bash that which python interpretter to use (`python3`) and where to find it (in the `/usr/bin/env folder`). It also says that the rest of the file is uses UTF-8 characters (used by English and similar languages).

The next several lines are a "doc comment" explaining ab bit about the provenance of the code. Sometimes this section may also include a license (e.g., GPL, MIT, BSD, public domain, etc.).  
```python
"""
File: course_schedules_tabula.py

Created on Tue Dec  5 12:57:56 2017

@author: chuntley

A utility for extracting Fairfield U course data from text scraped PDF files using tabula.
Currently works for the Spring 2018 Course Booklet.

"""
```
Doc comments are always triple-quoted and start on the first non # commented line of code. They are used by special software to generate documentation for developers. See an example [here in the Python docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).  

__3. Imports.__  
To make them easy to find, we always import external modules just after the top comments. 
```python
import re
import csv
import json
```
The modules above happen to be included in the standard library (i.e., vanilla Python). If a module were to come from a third-party library that has to be installed separately, then we would insert a comment with a `pip install` or `conda install` command. 

__4. Constants and Default Values.__
The next section provides a section of configuration data as variables that can be used later in the file. If the data is meant to be static then by convention we would use ALL_CAPS to indicate that it is a constant. If it is meant to be overridable, then we use a standard Python variable.
```python
# A set of tags that appear in the Notes field of a course_spec string
tags = {
  'CLRC':'Creative Life Residential College',
  'CORN':'Cornerstone Course',
  'HYBD':'Hybrid Course',
  'IGRC':'Ignatian Residential College',
  'RCOL':'Residential Colleges',
  'SERO':'Service Learning Option',
  'RNNU':'RN to BSN Students Only',
  'SJRC':'Service for Justice Residential College',
  'SDNU':'Second Degree Nurses Only',
  'UDIV':'U.S. Diversity',
  'SERL':'Service Learning',
  'WDIV':'World Diversity'
}

# A set of regular expressions (regex patterns) to use to extract data fields from a table row
flds = {
    'crn':re.compile('(^[0-9]+)'),
    'catalog_id':re.compile('(^[A-Z]+ [0-9,A-Z]+)'),
    'section':re.compile('(^[0-9,A-Z]+)'),
    'credits':re.compile('(^[0-9])'),
    'timecode':re.compile('(TBA|[Bb]y [Aa]rrangement|[Oo]nline|[MTWRFSU]+ [0-9]{4}-[0-9]{4}[PpAa][Mm])'),
    'tags':re.compile('('+'|'.join(tags.keys())+')'),
    'instructor':re.compile('(.+)'),
    'title':re.compile('(.+)')
}
```
In this case, `tags` and `flds` are designed to be overridden by any code that imports this module by setting `course_schedules_tabula.tags` or `course_schedules_tabula.flds` as needed. 

__5. The `scrape_undergrad_course_booklet()` function.__  
We'll need to skip over the `parse_table_row()` function to the bottom of the file, where we find the primary function of this module:
```python
def scrape_undergrad_course_booklet(filename):
    ''' Parse a course booklet that has been exported as a CSV from Tabula.'''
    with open(filename, newline='') as csvfile:
        linereader = csv.reader(csvfile)
        course_specs =[]
        for row in linereader:
            if not row[0].startswith('CRN'):
                course_spec = parse_table_row(row)
                if 'crn' in course_spec:
                    # add the new course_spec
                    course_specs += [course_spec]
                elif 'timecodes' in course_spec:
                    # merge timecode into last course_spec
                    course_specs[-1]['timecodes'] += course_spec['timecodes']
    return {'course_offerings':course_specs,'tags':tags}
```
The logic is actually pretty straightforward: 
1. Open the file as a CSV file using the standard `csv.reader()` function to break each line into columns.
2. After initializing a `course_specs` accumulator variable (as a blank list), read the CSV file one row at a time.
3. If the row is not a row of column headers then ...
    3a. Use the `parse_table_row()` function (just above it in the file) to create a `course_spec` for the course.
    3b. Handle two possible cases:
        - for normal rows (which have a `crn` key specified), `course_spec` is added to our accumulated list of `course_specs`
        - for extra timecode rows (which don't have a `crn` key but do have a `timecode` key), `course_spec` is merged into the last (previous) `course_spec` on our list. Note that the `'timecodes'` field is a list to allow a course to have more than one timecode. 
4. Return a dictionary with two items. The first item is for the `course_specs` we just created. The other is to provide a list of tags to look for in the notes.     

__6. The `parse_table_row()` Function.__
We'll take this one in slow motion. It's complicated. 

First, why is it above the `scrape_undergrad_course_booklet()` function in the file? Because `parse_table_row()` must exist (i.e., be loaded before) we can use it in `scrape_undergrad_course_booklet()`. Now lets see what it does. 

The top of the function definition specifies the calling signature (function name and parameters) and a doc comment explaining what the function does. Then on the next line it initializes `course_spec`, a dictionary that we will build incrementally as we parse the line into fields (columns), with one dictionary item per column.   
```python
def parse_table_row(row,date_range):
    ''' Parse one row of tabula data; each row is a column-wise list of strings'''
    
    course_spec = {}
```

The next bit of code deals with extra timecodes (Days + Time) that appear on a line by themselves. This is a common design, where easly detected special cases are handled first, with a `return` satement used to 'short-circuit' the function before it tries to handle the normal case. 
```python
    # Deal with extra timecodes on rows by themselves
    if not row[0]:
        unparsed = ' '.join(row)
        # use a regex to extract the timecode
        course_spec['timecodes'] = flds['timecode'].findall(unparsed)

        # return a partial course_spec with just the timecode
        return course_spec
```
The logic is a little tricky but still coherent if you trace it through one line at a time:
- `unparsed = ' '.join(row)` converts the columns back into a line of text with space separators. Huh? Well, if we can't trust the column breaks generated by Tabula then we might as well ignore them.
- `course_spec['timecodes'] = flds['timecode'].findall(unparsed)` uses a [regular expression](https://docs.python.org/2/library/re.html) to extract all timecodes found in the line of text. The specific regular expression is `'(TBA|[Bb]y [Aa]rrangement|[Oo]nline|[MTWRFSU]+ [0-9]{4}-[0-9]{4}[PpAa][Mm])'` as defined in the `flds` variable at the top of the file.   
- `return course_spec` short-circuits function execution, ending the function before the rest can be executed. Note that in this case `course_spec` is a dictionary with a single item: a list of timecodes. 

With the special case handled, the rest of the function deals with the normal case.
```python
    # What follows handles a typical table row exported from tabula

    # Parse out the easier columns that always seem to work in tabula
    course_spec['crn'] = int(row[0])
    course_spec['catalog_id'] = row[1] + ' ' + row[2]
    course_spec['section'] = row[3]
    course_spec['title'] = row[4]

    # Parse out the trickier columns that seem to merge awkwardly in tabula.
    # The logic below applies regular expressions to an unparsed string.
    # For each column:
    #   1. use a regex to extract data from the unparsed string;
    #   2. remove the extracted data from the unparsed string
    unparsed = ' '.join(row[5:]) # create a string of columns

    credits = flds['credits'].findall(unparsed)
    course_spec['credits'] = int(credits[0]) if credits else 0 # number of credits
    unparsed = flds['credits'].sub('',unparsed)

    course_spec['tags'] = flds['tags'].findall(unparsed) # list of tags
    unparsed = flds['tags'].sub('',unparsed)

    course_spec['timecodes']=flds['timecode'].findall(unparsed) # list of timecodes
    unparsed = flds['timecode'].sub('',unparsed)

    course_spec['instructor']=unparsed.strip() # remainder, minus extra whitespace

    return course_spec
```
For columns that never seem to get mangled by Tabula, we can just copy them directly into the dictionary:
```python
# Parse out the easier columns that always seem to work in tabula
    course_spec['crn'] = int(row[0])
    course_spec['catalog_id'] = row[1] + ' ' + row[2]
    course_spec['section'] = row[3]
    course_spec['title'] = row[4]
```
For the other columns we use the same trick as before, using regular expression pattern matches and a bit of validation logic to clean up missing data and bad dates.    

__7. A quick and dirty test.__  
The last line of the module is commented out. It was used to run the code directly (i.e., without a notebook or debugger) from a `bash` command line while it was being written. 
```python
# print(scrape_undergrad_course_booklet('tabula-201801CourseBooklet.csv','01/16-05/01'))
```
__8. Wrapping up: Running the code within a Notebook.__  
Since the code is in a module, we can call it directly within our notebooks. We can even feed it new data (e.g., for this semester) by setting the `filename` and `date_range`. Here is what it generates (which should look familiar to any practiced web developer as JSON, the defacto data standard for web service APIs) ...

In [1]:
import course_schedules_tabula
course_schedules_tabula.scrape_undergrad_course_booklet('tabula-201801CourseBooklet.csv')['course_offerings'][:200]

[{'crn': 34379,
  'catalog_id': 'AY 0010',
  'section': '01',
  'title': 'Intro Four-Field Anthropology',
  'credits': 3,
  'tags': ['WDIV'],
  'timecodes': ['TF 1100-1215pm'],
  'instructor': 'Lacy S'},
 {'crn': 34380,
  'catalog_id': 'AY 0010',
  'section': '02',
  'title': 'Intro Four-Field Anthropology',
  'credits': 3,
  'tags': ['WDIV'],
  'timecodes': ['TF 1230-0145pm'],
  'instructor': 'Lacy S'},
 {'crn': 35688,
  'catalog_id': 'AY 0052',
  'section': '01',
  'title': 'Culture and Political Economy',
  'credits': 3,
  'tags': [],
  'timecodes': ['TF 0930-1045am'],
  'instructor': 'Crawford D'},
 {'crn': 34553,
  'catalog_id': 'AY 0110',
  'section': '01',
  'title': 'Biological Anthropology',
  'credits': 3,
  'tags': [],
  'timecodes': ['W 0200-0430pm'],
  'instructor': 'Hensley-Marschand B'},
 {'crn': 34749,
  'catalog_id': 'AY 0110',
  'section': '02',
  'title': 'Biological Anthropology',
  'credits': 3,
  'tags': [],
  'timecodes': ['W 0630-0900pm'],
  'instructor': 'Hensl